In [ ]:
# Test functions for baseline_calculator_code.py and baseline_calculator_test.py

In [1]:
# Import test json script for testing
import json
import numpy as np
import baseline_calculator_code_2a
import os
import requests
from unittest.mock import patch, call
import unittest

# Reads json file and puts file information into dict form i.e. test data
#with open('test_json_traverse.json') as f:
    #test_data_dict = json.load(f)

## Outputs test_json_traverse file as a dictionary
#print(test_data_dict)

In [2]:
class Useful_json_translators(object):
    def __init__(self):
        ## Dict translators for building class, end use, and fuel type found on in the EIA data
        # Dict of building class
        self.bldg_class_translator = {'residential': 'RESD', 'commercial': 'COMM'}

        #Dict of end uses
        self.end_use_translator = {'clothes washers': 'CLW', 'clothes dryers': 'CDR', 'computers': 'CMPR', 'cooking': 'CGR', 'computing': 'OTHEQPPC' ,
        'delivered energy':'DELE', 'dishwashers': 'DSW', 'fans and pumps': 'FPR', 'freezers': 'FRZ', 'lighting': 'LGHTNG', 'office equipment': 'OTHEQPNPC',
        'other uses': 'OTHU','refrigeration': 'REFR', 'cooling': 'SPC', 'heating': 'SPH', 'TVs': 'TVR', 'ventilation': 'VNTC', 'water Heating': 'WTHT' }
        
        #Dict of fuel type, electricity is residential
        # purchased electricity is for commercial
        self.fuel_type_translator = {'electricity': 'ELC', 'Purchased Electricity': 'PRC', 'natural gas': 'NG'}
        
        ## Read in test data
        #with open('test_json_traverse.json') as f:
            #self.data_dict = json.load(f)
        
        ## Read in real data
        #with open('supporting_data/stock_energy_tech_data/mseg_res_com_cz.json') as f:
            #self.data_dict = json.load(f)

In [3]:
json_dicts = Useful_json_translators()

In [4]:
class Set_Up_Real_Data(object):
    """ Test downstream functions by importing real data """
    @classmethod
    def import_real_data(self):
        with open('supporting_data/stock_energy_tech_data/mseg_res_com_cz.json') as f:
            self.data_dict = json.load(f)
            return self.data_dict

In [5]:
real_data = Set_Up_Real_Data()

In [8]:
## Current
filter_strings = ['residential','electricity' , 'refrigeration']
def recursive(data_dict, filter_strings , energy_vals, position_list=[]):
    # OPERATION:   Recursive function traverses the json structure
    #              Grabs energy values per year as numpy array
    #              Test against test data first, then real data

    ## Initialize classes
    json_dicts = Useful_json_translators()
    #real_data = Set_Up_Real_Data()


    for key, value in real_data.data_dict.items():
        if isinstance(value, dict):  
            recursive(value, filter_strings , energy_vals, position_list+[key]) 
            
        else:
        ## At the endpoint/leaf node, use relevant fuel type and end use combination to filter neccessary energy values, aggregate energy value by year
            
            ## Store each year as a list
            years_as_keys_list = (position_list+[key])
            
            ## Convert list into numpy array
            years_as_keys_array = np.array(years_as_keys_list)

            ## Create a dict for residential and commercial bldg types 
            ## Check format in EIA for proper syntax matching, 
            all_bldg_types ={'residential': ['single family home', 'multi family home', 'mobile homes'], 
            'commercial': ['assembly', 'education', 'food sales','food service', 'health care', 'lodging',
            'small office','large office','mercantile/serice', 'warehouse', 'other']}
            
            if filter_strings[1] == years_as_keys_list[2] and filter_strings[2] == years_as_keys_list[3] and all_bldg_types[filter_strings[0]] and 'energy' in years_as_keys_list:
                ## Sort by keys in year order and grab values in the order of the years as numpy array
                energy_vals+=np.array([real_data.data_dict.keys() for key,value in sorted(real_data.data_dict.items())])
                ## De-bugging purposes
                print(type(energy_vals))

    return (energy_vals)

In [10]:
filter_strings = ['residential','electricity' , 'refrigeration']
recursive(real_data.data_dict, filter_strings, np.zeros(2))

AttributeError: 'Set_Up_Real_Data' object has no attribute 'data_dict'

In [ ]:
def recursive_prev(data_dict, filter_strings , energy_vals, position_list=[]):
    # OPERATION:   Recursive function traverses the json structure
    #              Grabs energy values per year as numpy array
    #              Test against test data first, then real data

    ## Initialize json translators
    #json_dicts = Useful_json_translators()

    for key, value in test_data_dict.items():
        if isinstance(value, dict):  
            recursive_prev(value, filter_strings , energy_vals, position_list+[key]) 
            
        else:
        ## At the endpoint/leaf node, use relevant fuel type and end use combination to filter neccessary energy values, aggregate energy value by year
            
            ## Store each year as a list
            years_as_keys_list = (position_list+[key])
            
            ## Convert list into numpy array
            years_as_keys_array = np.array(years_as_keys_list)

            ## Create a dict for residential and commercial bldg types 
            ## Check format in EIA for proper syntax matching, 
            all_bldg_types ={'residential': ['single family home', 'multi family home', 'mobile homes'], 
            'commercial': ['assembly', 'education', 'food sales','food service', 'health care', 'lodging',
            'small office','large office','mercantile/serice', 'warehouse', 'other']}
            
            if filter_strings[1] == years_as_keys_list[2] and filter_strings[2] == years_as_keys_list[3] and all_bldg_types[filter_strings[0]] and 'energy' in years_as_keys_list:
                ## Sort by keys in year order and grab values in the order of the years as numpy array
                energy_vals+=np.array([test_data_dict[key] for key,value in sorted(test_data_dict.items())])
                ## De-bugging purposes
                #print(type(energy_vals))

    return (energy_vals)


In [ ]:
filter_strings = ['residential', 'refrigeration']
recursive_prev(test_data_dict, filter_strings, np.zeros(2))

In [ ]:
def construct_EIA_series_ID(filter_strings):
    ## Initialize json translators
    json_dicts = Useful_json_translators()

    ## this statement will run if outcome is true
    if filter_strings[0] == 'residential':
        condition_1 = json_dicts.end_use_translator[filter_strings[2]]
        condition_2 = 'NA'

    ## Will run when 1st if block is false
    elif filter_strings[0] == 'commercial':
        condition_1 = 'NA'
        condition_2 = json_dicts.end_use_translator[filter_strings[2]]

    ## EIA series ID string
    eia_series_ID = 'AEO.2020.REF2020.CNSM_NA_'+json_dicts.bldg_class_translator[filter_strings[0]]+'_'+condition_1+'_'+json_dicts.fuel_type_translator[filter_strings[1]]+'_'+condition_2+'_USA_QBTU.A'
    ## De-bugging purposes
    #print(eia_series_ID)
    return (eia_series_ID)

In [ ]:
def data_comparsion(data_dict,filter_strings, np_array):
    ## OPERATION:   Data comparsion function attempts to run recursive function 
    ##              Traverses real data, calls EIA API, with filter strings list comprehension
    ##              If succesful, compares energy values from
    ##              traversed real data and EIA API
    ##              If unsuccessful, has IndexError, prints message


    ## Initialize json translators
    json_dicts = Useful_json_translators()
    ## If no exception occurs, then except is skipped
    ## If exception occurs, then rest of try block is skipped
    try:
        ## call recursive function, get energy values, change filter string to match the fiter string combo you call from recursive
        recursive(json_dicts.data_dict, filter_strings, np.zeros(2))
        energy_vals = recursive(json_dicts.data_dict, filter_strings, np.zeros(2))

        ## Call construct EIA series ID function, if 2 tuples enter, stops here, triggers IndexError
        eia_str_series_ID = construct_EIA_series_ID(filter_strings)

        ## call EIA API and grab EIA energy values based on years
        eia_data_array, eia_data_years = data_processor(api_query(os.environ['EIA_API_KEY'], eia_str_series_ID))
        #print(type(eia_data_array))
    
        ## sum EIA values, compare to json energy value
        eia_energy_sum = sum(eia_data_array)
        
        ## De-bugging purposes
        #print(eia_energy_sum)
        ## else body runs when if statement is false, but don't need this cause have except block
        
    ## Only runs if try block is unsuccessful
    except IndexError:
        print('Incorrect number of index, check:', filter_strings)
    except AttributeError:
        print('check filter strings list', filter_strings)
    except (ValueError, KeyError):
        print('Data not present in json, EIA API not accessed. Review filter strings:', filter_strings)
    ## Using bare except to catch all errors
    #except:
        #print('Some other error occurred')
    
    ## If try block successful, then this runs
    else:
        return (eia_energy_sum, energy_vals)

In [ ]:
## Current function-
def recursive_new(data_dict, filter_strings , energy_vals, position_list=[]):
    ## Initialize json translators
    json_dicts = Useful_json_translators()
    
    ## Create a dict for residential and commercial bldg types 
    ## Check format in EIA for proper syntax matching, 
    all_bldg_types ={'residential': ['single family home', 'multi family home', 'mobile homes'], 
    'commercial': ['assembly', 'education', 'food sales','food service', 'health care', 'lodging',
    'small office','large office','mercantile/serice', 'warehouse', 'other']}
    
    for key, value in json_dicts.data_dict.items():
        ## if this is true, then keep going traversing through json
        if isinstance(value, dict):
            try:  
                recursive(value, filter_strings , energy_vals, position_list+[key])
        
        ## if 1st statement false, then.. 
        elif filter_strings[1] == years_as_keys_list[2] and filter_strings[2] == years_as_keys_list[3] and all_bldg_types[filter_strings[0]] and 'energy' in years_as_keys_list:
            ## Sort by keys in year order and grab values in the order of the years as numpy array
            energy_vals+=np.array([json_dicts.data_dict[key] for key,value in sorted(json_dicts.data_dict.items())])
            ## De-bugging purposes
            #print(type(energy_vals))
            
            ## Store each year as a list
            years_as_keys_list = (position_list+[key])
            
            ## Convert list into numpy array
            years_as_keys_array = np.array(years_as_keys_list)

             
            

    return (energy_vals)


In [ ]:
## New function

def recursive_new(data_dict, filter_strings , energy_vals, position_list=[]):
    for key, value in json_dicts.data_dict.items():
        if isinstance(value, dict):
            try:
                recursive(value, filter_strings , energy_vals, position_list+[key])
        
                ## Store each year as a list
                years_as_keys_list = (position_list+[key])
                #print(years_as_keys_list)

                ## Convert list into numpy array
                years_as_keys_array = np.array(years_as_keys_list)

                ## Create a dict for residential and commercial bldg types 
                ## Check format in EIA for proper syntax matching, 
                all_bldg_types ={'residential': ['single family home', 'multi family home', 'mobile homes'], 
                'commercial': ['assembly', 'education', 'food sales','food service', 'health care', 'lodging',
                'small office','large office','mercantile/serice', 'warehouse', 'other']}

                if filter_strings[1] == years_as_keys_list[2] and filter_strings[2] == years_as_keys_list[3] and all_bldg_types[filter_strings[0]] and 'energy' in years_as_keys_list:
                    ## Sort by keys in year order and grab values in the order of the years as numpy array
                    energy_vals+=np.array([json_dicts.data_dict[key] for key,value in sorted(json_dicts.data_dict.items())]
            except:
                print('Error occurred')
            

    return (energy_vals)

In [ ]:
## Break into pieces to test
filter_strings = ['residential', 'refrigeration']

for key, value in json_dicts.data_dict.items():
    if isinstance(value, dict):  
        recursive(value, filter_strings , energy_vals, position_list+[key]) 
    else:
        years_as_keys_list = (position_list+[key])
        
        ## Create a dict for residential and commercial bldg types 
        ## Check format in EIA for proper syntax matching, 
        all_bldg_types ={'residential': ['single family home', 'multi family home', 'mobile homes'], 
        'commercial': ['assembly', 'education', 'food sales','food service', 'health care', 'lodging',
        'small office','large office','mercantile/serice', 'warehouse', 'other']}

        if filter_strings[1] == years_as_keys_list[2] and filter_strings[2] == years_as_keys_list[3] and all_bldg_types[filter_strings[0]] and 'energy' in years_as_keys_list:
            ## Sort by keys in year order and grab values in the order of the years as numpy array
            energy_vals+=np.array([data_dict[key] for key,value in sorted(data_dict.items())])
            print(energy_vals)

In [ ]:
## Current function-9/22/20
def data_comparsion(data_dict,filter_strings, np_array):
    ## Initialize json translators
    json_dicts = Useful_json_translators()
    ## If no exception occurs, then except is skipped
    ## If exception occurs, then rest of try block is skipped
    try:
        ## call recursive function, get energy values, change filter string to match the fiter string combo you call from recursive
        recursive(json_dicts.data_dict, filter_strings, np.zeros(2))
        energy_vals = recursive(json_dicts.data_dict, filter_strings, np.zeros(2))

        ## Call construct EIA series ID function
        eia_str_series_ID = construct_EIA_series_ID(filter_strings)

        ## call EIA API and grab EIA energy values based on years
        eia_data_array, eia_data_years = data_processor(api_query(os.environ['EIA_API_KEY'], eia_str_series_ID))
        #print(type(eia_data_array))
    
        ## sum EIA values, compare to json energy value
        eia_energy_sum = sum(eia_data_array)
        
        ## De-bugging purposes
        #print(eia_energy_sum)
        ## else body runs when if statement is false, but don't need this cause have except block
        
    ## Only runs if try block is unsuccessful
    except IndexError:
        print('Incorrect number of index, check:', filter_strings)
    except AttributeError:
        print('check filter strings list', filter_strings)
    except (ValueError, KeyError):
        print('Data not present in json, EIA API not accessed. Review filter strings:', filter_strings)
    ## Using bare except to catch all errors
    #except:
        #print('Some other error occurred')
    
    ## If try block successful, then this runs
    else:
        return (eia_energy_sum, energy_vals)

In [ ]:
## Testing test I wrote for except block, what happens when input 2 tuples, 
## Should trigger an IndexError

# Purposely input only bldg class and end use for input into
# recursive function to trigger exception
filter_str_exception = ['residential','refrigeration']

recursive(json_dicts.data_dict, filter_str_exception, np.zeros(2))

In [ ]:
def construct_EIA_series_ID(filter_strings):
    ## Initialize json translators
    json_dicts = Useful_json_translators()

    ## this statement will run if outcome is true
    if filter_strings[0] == 'residential':
        condition_1 = json_dicts.end_use_translator[filter_strings[2]]
        condition_2 = 'NA'

    ## Will run when 1st if block is false
    elif filter_strings[0] == 'commercial':
        condition_1 = 'NA'
        condition_2 = json_dicts.end_use_translator[filter_strings[2]]

    ## EIA series ID string
    eia_series_ID = 'AEO.2020.REF2020.CNSM_NA_'+json_dicts.bldg_class_translator[filter_strings[0]]+'_'+condition_1+'_'+json_dicts.fuel_type_translator[filter_strings[1]]+'_'+condition_2+'_USA_QBTU.A'
    print(eia_series_ID)

In [ ]:
## What happens when inserts a 2 tuple for filter strings, instead 3
eia_str_series_ID = construct_EIA_series_ID(filter_str_exception)

In [ ]:
## Call data comparsion with 2 tuples
data_comparsion(json_dicts.data_dict, filter_str_exception, (np.zeros(2), np.zeros(2)))    

In [ ]:
## Break test into pieces
filter_str_exception = ['residential','refrigeration']

mock_print = baseline_calculator_code_2a.data_comparsion(json_dicts.data_dict, filter_str_exception, (np.zeros(2), np.zeros(2)))
## De-bugging purposes
#print(mock_print)



In [ ]:
## Testing construct EIA_series_ID function
filter_strings = ['commercial', 'natural gas', 'cooking']
expected_outcome = 'AEO.2020.REF2020.CNSM_NA_COMM_NA_NG_CGR_USA_QBTU.A'

#condition_1 = 'NA'
#condition_2 = json_dicts.end_use_translator[filter_strings[2]]
    

eia_str_test = construct_EIA_series_ID(filter_strings)

In [ ]:
## Function to call EIA API
def api_query(api_key, series_id):
    """Execute an EIA API query and extract the data returned

    Execute a query of the EIA API and extract the data from the
    structure returned by the API.

    Args:
        api_key (str): EIA API key.
        series_id (str): Identifying string for a specific data series.

    Returns:
        A nested list of data with inner lists structured as
        [year string, data value].
    """
    query_str = ('http://api.eia.gov/series/?api_key='+ api_key +
                 '&series_id=' + series_id)
    data = requests.get(query_str).json()
    try:
        data = data['series'][0]['data']
    # If an invalid series_id is used, the 'series' key will not be present
    except KeyError:
        print('\nSeries ID not available from API: ' + series_id)
    return data




In [ ]:
## Format data received by EIA API
def data_processor(data):
    """Restructure the data obtained from the API into numpy arrays

    Args:
        data (list): A list of data, with each value a list where
            the first value is the year as a string and the second
            value is the number for that year for the data requested.

    Returns:
        Two numpy arrays, one containing the data and one containing
        the years corresponding to those data. Both arrays are sorted
        according to ascending year order.
    """
    years, data = zip(*data)
    years = np.array(years)
    data = np.array(data)[years.argsort()]  # Re-sort in ascending year order
    years = years[years.argsort()]  # Re-sort to be in ascending year order
    return data, years

In [ ]:
## LATEST -8/17 Recursive function
def recursive(data_dict, filter_strings , energy_vals, position_list=[]):
    #with open('test_json_traverse.json') as f:
        #data_dict = json.load(f)

    for key, value in data_dict.items():
        if isinstance(value, dict):  
            recursive(value, filter_strings , energy_vals, position_list+[key]) 
            
        else:
        ## At the endpoint/leaf node, use relevant fuel type and end use combination to filter neccessary energy values, aggregate energy value by year
            
            ## Store each year as a list
            years_as_keys_list = (position_list+[key])
            #print(years_as_keys_list)
            
            ## Convert list into numpy array
            years_as_keys_array = np.array(years_as_keys_list)

            ## Create a dict for residential and commercial bldg types 
            ## Check format in EIA for proper syntax matching, 
            all_bldg_types ={'residential': ['single family home', 'multi family home', 'mobile homes'], 
            'commercial': ['assembly', 'education', 'food sales','food service', 'health care', 'lodging',
            'small office','large office','mercantile/serice', 'warehouse', 'other']}
            
            if filter_strings[1] == years_as_keys_list[2] and filter_strings[2] == years_as_keys_list[3] and all_bldg_types[filter_strings[0]] and 'energy' in years_as_keys_list:
            #if filter_strings[1] and filter_strings[2] and all_bldg_types[filter_strings[0]] and 'energy' in years_as_keys_list:
                ## Sort by keys in year order and grab values in the order of the years as numpy array
                energy_vals+=np.array([data_dict[key] for key,value in sorted(data_dict.items())])
                ## De-bugging purposes
                #print(years_as_keys_list)
                #print(energy_vals)

    return (energy_vals)

In [ ]:
def data_comparsion_now(data_dict,filter_strings, np_array):
    ## Initialize json translators
    json_dicts = Useful_json_translators()
    
    try:
        # call recursive function, get energy values, change filter string to match the fiter string combo you call from recursive
        recursive(json_dicts.data_dict, filter_strings, np.zeros(2))
        energy_vals = recursive(json_dicts.data_dict, filter_strings, np.zeros(2))
        
        ## this statement will run if outcome is true
        if filter_strings[0] == 'residential':
            condition_1 = json_dicts.end_use_translator[filter_strings[2]]
            condition_2 = 'NA'
        
        ## Will run when 1st if block is false
        elif filter_strings[0] == 'commercial':
            condition_1 = 'NA'
            condition_2 = json_dicts.end_use_translator[filter_strings[2]]

        ## EIA series ID string
        eia_series_ID = 'AEO.2020.REF2020.CNSM_NA_'+json_dicts.bldg_class_translator[filter_strings[0]]+'_'+condition_1+'_'+json_dicts.fuel_type_translator[filter_strings[1]]+'_'+condition_2+'_USA_QBTU.A'
        
        ## call EIA API and grab EIA energy values based on years
        eia_data_array, eia_data_years = data_processor(api_query(os.environ['EIA_API_KEY'], eia_series_ID))
        #print(type(eia_data_array))
        
        ## sum EIA values, compare to json energy value
        eia_energy_sum = sum(eia_data_array)
        
        ## De-bugging purposes
        #print(eia_energy_sum)
        ## else body runs when if statement is false, but don't need this cause have except block
        
    ## Only runs if try block is unsuccessful
    except IndexError:
        print('filter strings list index out of range')
    except AttributeError:
        print('check filter strings list')
    except KeyError:
        print('Series ID not available on EIA API')
    ## Using bare except to catch all errors
    except:
        print('Some other error occurred')
    
    ## If try block successful, then this runs
    else:
        return (eia_energy_sum, energy_vals)

In [ ]:
## Testing new modified/simplified data comparsion fun - 9/21/20
#filter_strings = ['residential','electricity', 'refrigeration']
filter_strings =['commercial', 'natural gas', 'cooking']

test = data_comparsion_now(json_dicts.data_dict, filter_strings, (np.zeros(2), np.zeros(2)))
print(test)

In [ ]:
## Using data comparsion function that works 
def data_comparsion_1(data_dict,filter_strings, np_array):
    ## Initialize json translators
    json_dicts = Useful_json_translators()
    
    try:
        # call recursive function, get energy values, change filter string to match the fiter string combo you call from recursive
        recursive(json_dicts.data_dict, filter_strings, np.zeros(2))
        energy_vals = recursive(json_dicts.data_dict, filter_strings, np.zeros(2))

    ## Only runs if try block is unsuccessful
    except IndexError:
        print('filter strings list index out of range')
    except AttributeError:
        print(" 'NoneType' object has no attribute 'mock_calls' ")
    ## Using bare except to catch all errors
    except:
        print('Some other error occurred')

    ## If try block successful, then this runs
    else:
        ## Constructed EIA API series ID string if residential building
        if filter_strings[0] == 'residential':
            condition_1 = json_dicts.end_use_translator[filter_strings[2]]
            condition_2 = 'NA'

            ## EIA series ID string
            eia_series_ID = 'AEO.2020.REF2020.CNSM_NA_'+json_dicts.bldg_class_translator[filter_strings[0]]+'_'+condition_1+'_'+json_dicts.fuel_type_translator[filter_strings[1]]+'_'+condition_2+'_USA_QBTU.A'

            ## call EIA API and grab EIA energy values based on years
            eia_data_array, eia_data_years = data_processor(api_query(os.environ['EIA_API_KEY'], eia_series_ID))
            #print(type(eia_data_array))

            ## sum EIA values, compare to json energy value
            eia_energy_sum = sum(eia_data_array)

        # Constructed EIA API series ID string if commercial building
        if filter_strings[0] == 'commercial':
            condition_1 = 'NA'
            condition_2 = json_dicts.end_use_translator[filter_strings[2]]

            ## EIA series ID string
            eia_series_ID = 'AEO.2020.REF2020.CNSM_NA_'+json_dicts.bldg_class_translator[filter_strings[0]]+'_'+condition_1+'_'+json_dicts.fuel_type_translator[filter_strings[1]]+'_'+condition_2+'_USA_QBTU.A'

            ## call EIA API and grab EIA energy values based on years
            eia_data_array, eia_data_years = data_processor(api_query(os.environ['EIA_API_KEY'], eia_series_ID))
            #print(type(eia_data_array))

            ## sum EIA values, compare to json energy value
            eia_energy_sum = sum(eia_data_array)

        return (eia_energy_sum, energy_vals)
            

In [ ]:
## TESTING: Call data comparsion function with two tuples and see if I get index error
##data_comparsion_1 is currently working
filter_strings = ['residential','electricity', 'refrigeration']
#filter_strings =['commercial', 'natural gas', 'cooking']

json_dicts = Useful_json_translators()

test_1 = data_comparsion_1(json_dicts.data_dict,filter_strings, (np.zeros(2), np.zeros(2)))
print(test_1)


In [ ]:
## Testing logic for series ID

json_dicts = Useful_json_translators()

filter_strings =['commercial', 'natural gas', 'cooking']
#filter_strings = ['residential','electricity', 'refrigeration']

## call recursive function, get energy values, change filter string to match the fiter string combo you call from recursive
recursive(json_dicts.data_dict, filter_strings, np.zeros(2))
energy_vals = recursive(json_dicts.data_dict, filter_strings, np.zeros(2))

## this statement will run if outcome is true
if filter_strings[0] == 'residential':
    condition_1 = json_dicts.end_use_translator[filter_strings[2]]
    condition_2 = 'NA'

## Will run when 1st if block is false
elif filter_strings[0] == 'commercial':
    condition_1 = 'NA'
    condition_2 = json_dicts.end_use_translator[filter_strings[2]]

## EIA series ID string
eia_series_ID = 'AEO.2020.REF2020.CNSM_NA_'+json_dicts.bldg_class_translator[filter_strings[0]]+'_'+condition_1+'_'+json_dicts.fuel_type_translator[filter_strings[1]]+'_'+condition_2+'_USA_QBTU.A'

## call EIA API and grab EIA energy values based on years
eia_data_array, eia_data_years = data_processor(api_query(os.environ['EIA_API_KEY'], eia_series_ID))
#print(type(eia_data_array))

## sum EIA values, compare to json energy value
eia_energy_sum = sum(eia_data_array)
print(eia_energy_sum)



In [ ]:
filter_strings = ['residential','electricity', 'refrigeration']
#filter_strings =['commercial', 'natural gas', 'cooking']

json_dicts = Useful_json_translators()

test_2 = data_comparsion_1(json_dicts.data_dict,filter_strings, (np.zeros(2), np.zeros(2)))
print(test_2)



In [ ]:
class test_data_comparsion_expect_block(unittest.TestCase):
    # Purposely input only bldg class and end use for input into
    # recursive function to trigger exception
    filter_str_exception = ['residential','refrigeration']

    # Text expected to be printed to the stdout by the except block
    expected_except_print = ('list index out of range' )

    @patch('builtins.print')
    def test_except_print(self, mock_print):
        json_dicts = Useful_json_translators()

        mock_print = baseline_calculator_code_2a.data_comparsion(
            json_dicts.data_dict, self.filter_str_exception, np.zeros(2))

        ## De-bugging purposes
        print(mock_print)


In [ ]:
@patch('builtins.print')
def test_except_print(self, mock_print):
    json_dicts = Useful_json_translators()
    filter_str_exception = ['residential','refrigeration']
    
    mock_print = baseline_calculator_code_2a.data_comparsion(
        json_dicts.data_dict, filter_str_exception, np.zeros(2))

    ## De-bugging purposes
    print(mock_print)



In [ ]:
def data_comparsion(data_dict,filter_strings, np_array):
     def __init__(self):
        ## Matches data in json file for Scout, fix format error
        self.json_data_translator_dict = {'residential': ['single family home', 'multi family home', 'mobile homes'],
                                 'electricity': ['heating','secondary heating' ,'cooling' ,'cooking','computers',
                                                 'drying','lighting', 'refrigeration','ceiling fan','fans and pumps',
                                                 'water heating','other', 'all'],
                                 'natural gas': ['heating','secondary heating', 'cooling', 'cooking', 'drying',
                                                 'water heating','other', 'all'],
                                 'distillate': ['heating', 'secondary heating', 'water heating', 'other', 'all'],
                                 'other fuel': ['heating', 'secondary heating','water heating', 'cooking', 'drying',
                                               'other', 'all'],
                                 'commercial': ['assembly', 'education', 'food sales','food service', 'health care',
                                                'lodging','small office','large office','mercantile/serice',
                                                'warehouse', 'other'], 
                                 'electricity': ['heating', 'cooling', 'ventilation', 'water heating','lighting',
                                                'refrigeration', 'cooking','PCs', 'non-PC office equipment', 'MELs','all'],
                                 'natural gas':['heating', 'cooling', 'water heating', 'cooking', 'all'],
                                 'distillate': ['heating', 'water heating', 'all']}

        ## Dict translators for building class, end use, and fuel type found on in the EIA data
        # Dict of building class
        self.bldg_class_translator = {'residential': 'RESD', 'commercial': 'COMM'}

        #Dict of end uses
        self.end_use_translator = {'clothes washers': 'CLW', 'clothes dryers': 'CDR', 'computers': 'CMPR', 'cooking': 'CGR', 'computing': 'OTHEQPPC' ,
        'delivered energy':'DELE', 'dishwashers': 'DSW', 'fans and pumps': 'FPR', 'freezers': 'FRZ', 'lighting': 'LGHTNG', 'office equipment': 'OTHEQPNPC',
        'other uses': 'OTHU','refrigeration': 'REFR', 'cooling': 'SPC', 'heating': 'SPH', 'TVs': 'TVR', 'ventilation': 'VNTC', 'water Heating': 'WTHT' }

        #Dict of fuel type, electricity is residential
        # purchased electricity is for commercial
        self.fuel_type_translator = {'electricity': 'ELC', 'Purchased Electricity': 'PRC', 'natural gas': 'NG'}

        ## Read in test data
        with open('test_json_traverse.json') as f:
            self.data_dict = json.load(f)

        try:
            #with open('supporting_data/stock_energy_tech_data/mseg_res_com_cz.json') as f:
                #data_dict = json.load(f)
                # call recursive function, get energy values, change filter string to match the fiter string combo you call from recursive
                recursive(self.data_dict, filter_strings, np.zeros(2))
                energy_vals = recursive(self.data_dict, filter_strings, np.zeros(2))

                ## Constructed EIA API series ID string if residential building
                if filter_strings[0] == 'residential':
                    condiiton_1 = self.end_use_translator[filter_strings[2]]
                    condition_2 = 'NA'

                    ## EIA series ID string
                    eia_series_ID = 'AEO.2020.REF2020.CNSM_NA_'+self.bldg_class_translator[filter_strings[0]]+'_'+condition_1+'_'+self.fuel_type_translator[filter_strings[1]]+'_'+condition_2+'_USA_QBTU.A'

                    ## call EIA API and grab EIA energy values based on years
                    eia_data_array, eia_data_years = data_processor(api_query(os.environ['EIA_API_KEY'], eia_series_ID))

                    ## sum EIA values, compare to json energy value
                    eia_energy_sum = sum(eia_data_array)
                return eia_data_array, energy_vals

                # Constructed EIA API series ID string if commercial building
                if filter_strings[0] == 'commercial':
                    condiiton_1 = 'NA'
                    condition_2 = self.end_use_translator[filter_strings[2]]

                    ## Constructed EIA series ID string
                    eia_series_ID = 'AEO.2020.REF2020.CNSM_NA_'+self.bldg_class_translator[filter_strings[0]]+'_'+condition_1+'_'+self.fuel_type_translator[filter_strings[1]]+'_'+condition_2+'_USA_QBTU.A'

                    ## call EIA API and grab EIA energy values based on years
                    eia_data_array, eia_data_years = data_processor(api_query(os.environ['EIA_API_KEY'], eia_series_ID))
                    #print(type(eia_data_array))
                    
                    ## sum EIA values, compare to json energy value
                    eia_energy_sum = sum(eia_data_array)
                    #return eia_data_array, energy_vals

        ## Only runs if try block is unsuccessful
        except IndexError:
            print('filter strings list index out of range')
        except AttributeError:
            print(" 'NoneType' object has no attribute 'mock_calls' ")
        ## Using bare except to catch all errors
        except:
            print('Some other error occurred')


In [ ]:
filter_strings = ['residential','electricity', 'refrigeration']
with open('test_json_traverse.json') as f:
    data_dict = json.load(f)

In [ ]:
test_2 = data_comparsion(data_dict, filter_strings, (np.zeros(2), np.zeros(2)))
print(test_2)

In [ ]:
## This is the real data, possibly issue why I am getting NONE, 
## maybe the filter string combo I am testing doesn't exist in the real data

#with open('supporting_data/stock_energy_tech_data/mseg_res_com_cz.json') as f:
    #data_dict = json.load(f)
        
## Try with test data and see what result is, if this doesn't fix the problem
with open('test_json_traverse.json') as f:
    data_dict = json.load(f)
    
## THen try patching in test data while keeping real data in code
## Make sure the patch return value is ... a value from the test file

In [ ]:
filter_strings = ['residential','refrigeration', 'electricity']
test = baseline_calculator_code_2a.data_comparsion(data_dict, filter_strings, (np.zeros(2), np.zeros(2)))
print(test)


In [ ]:
## Testing -9/15
class Useful_json_translators(object):
    def __init__(self):
        ## Dict translators for building class, end use, and fuel type found on in the EIA data
        # Dict of building class
        self.bldg_class_translator = {'residential': 'RESD', 'commercial': 'COMM'}

        #Dict of end uses
        self.end_use_translator = {'clothes washers': 'CLW', 'clothes dryers': 'CDR', 'computers': 'CMPR', 'cooking': 'CGR', 'computing': 'OTHEQPPC' ,
        'delivered energy':'DELE', 'dishwashers': 'DSW', 'fans and pumps': 'FPR', 'freezers': 'FRZ', 'lighting': 'LGHTNG', 'office equipment': 'OTHEQPNPC',
        'other uses': 'OTHU','refrigeration': 'REFR', 'cooling': 'SPC', 'heating': 'SPH', 'TVs': 'TVR', 'ventilation': 'VNTC', 'water Heating': 'WTHT' }
        
        #Dict of fuel type, electricity is residential
        # purchased electricity is for commercial
        self.fuel_type_translator = {'electricity': 'ELC', 'Purchased Electricity': 'PRC', 'natural gas': 'NG'}


In [ ]:
class test_differ_EIA_series_IDs(unittest.TestCase):
    ## Test different bldg class and fuel type, same end use
    ## Currently, testing: commercial and residential cooking and space heating

    ## Commercial cooking series name and expected series id, natural gas
    comm_cook_series_name = ['commercial', 'natural gas', 'cooking']
    expected_comm_cook_series_id = 'AEO.2020.REF2020.CNSM_NA_COMM_NA_NG_CGR_USA_QBTU.A'

    ## Residential cooking series name, electricity
    ## https://www.eia.gov/opendata/qb.php?category=3604376&sdid=AEO.2020.REF2020.CNSM_NA_RESD_CGR_ELC_NA_USA_QBTU.A
    #resd_cook_series_name = ['residential', 'electricity', 'cooking']
    #expected_resd_cook_series_id = 'AEO.2020.REF2020.CNSM_NA_RESD_CGR_ELC_NA_USA_QBTU.A'

    ## Commercial heating series name and expected series id, electricity
    #comm_spheat_series_name = ['commercial', 'electricity', 'heating']
    #expected_comm_spheat_series_id = 'AEO.2020.REF2020.CNSM_NA_COMM_NA_PRC_SPH_USA_QBTU.A'

    ## Residental, eletricity, space heating
    ## https://www.eia.gov/opendata/qb.php?category=3604376&sdid=AEO.2020.REF2020.CNSM_HHD_RESD_SPH_ELC_NA_USA_QBTU.A
    #resd_spheat_series_name = ['residential', 'electricity', 'heating']
    #expected_resd_spheat_series_id = 'AEO.2020.REF2020.CNSM_HHD_RESD_SPH_ELC_NA_USA_QBTU.A'

    def test_comm_cooking_str(self):
        ## Initialize json translators
        json_dicts = Useful_json_translators()

        ## Test created series ID, use line 201-202
        if self.comm_cook_series_name[0] == 'residential':
            condiiton_1 = json_dicts.end_use_translator[self.comm_cook_series_name[2]]
            condition_2 = 'NA'
            
            ## Constructed EIA series ID string
            eia_series_ID = 'AEO.2020.REF2020.CNSM_NA_'+json_dicts.bldg_class_translator[self.comm_cook_series_name[0]]
            +'_'+condition_1+'_'+json_dicts.fuel_type_translator[self.comm_cook_series_name[1]]+'_'+condition_2+'_USA_QBTU.A'

        if self.comm_cook_series_name[0] == 'commercial':
            condition_1 = 'NA'
            condition_2 = json_dicts.end_use_translator[self.comm_cook_series_name[2]]
            
            ## Constructed EIA series ID string
            eia_series_ID = 'AEO.2020.REF2020.CNSM_NA_'+json_dicts.bldg_class_translator[self.comm_cook_series_name[0]]
            +'_'+condition_1+'_'+json_dicts.fuel_type_translator[self.comm_cook_series_name[1]]+'_'+condition_2+'_USA_QBTU.A'

        ## Test if the constructed series_id for commercial strings matches expected EIA API
        self.assertEqual(eia_series_ID, self.expected_comm_cook_series_id)


In [ ]:
## Break up class above to test error
comm_cook_series_name = ['commercial', 'natural gas', 'cooking']
expected_comm_cook_series_id = 'AEO.2020.REF2020.CNSM_NA_COMM_NA_NG_CGR_USA_QBTU.A'
json_dicts = Useful_json_translators()


In [ ]:
if comm_cook_series_name[0] == 'commercial':
            condition_1 = 'NA'
            condition_2 = json_dicts.end_use_translator[comm_cook_series_name[2]]
            
            ## Constructed EIA series ID string
            eia_series_ID = 'AEO.2020.REF2020.CNSM_NA_'+json_dicts.bldg_class_translator[comm_cook_series_name[0]]+'_'+condition_1+'_'+json_dicts.fuel_type_translator[comm_cook_series_name[1]]+'_'+condition_2+'_USA_QBTU.A'



In [ ]:
## Testing second error- 9/15
## Code from test script
filter_str_exception = ['residential','refrigeration', 'electricity']

# Text expected to be printed to the stdout by the except block
expected_except_print = ('list index out of range' )

## Read in test data
with open('test_json_traverse.json') as f:
    data_dict = json.load(f)

    @patch('builtins.print')
    def test_except_print(self, mock_print):
        mock_print = baseline_calculator_code_2a.data_comparsion(
            self.data_dict, self.filter_str_exception, np.zeros(2))

        ## De-bugging purposes
        print(mock_print)

        ## Test that printed output from the except block matches the expected string
        ## Only catches raised error if its located inside its corresponding try block
        #self.assertEqual(mock_print, self.expected_except_print)
            #mock_print.mock_calls[0][1][0], self.expected_except_print)



In [ ]:
filter_str_exception = ['residential', 'electricity']
mock_print = baseline_calculator_code_2a.data_comparsion(data_dict, filter_str_exception, np.zeros(2))
print(mock_print)




In [ ]:
## View for de-bugging
eia_base_str

In [ ]:
filter_strings[0]

In [ ]:
## Create two conditions for cooking end use different EIA API series ID
## If end use is cooking, then create what is condition 1
## Condition 1: EIA series ID for commercial cooking has NA after bldg class and 
if filter_strings[2] == 'cooking' and filter_strings[0] == 'commercial':
    condition_1 ='NA'
    condition_2 = end_use_translator[filter_strings[2]]
    
    ## Want to convert this to generic form, maybe change back to eia_str
    eia_base_str = 'AEO.2020.REF2020.CNSM_NA_'+bldg_class_translator[filter_strings[0]]+'_'+condition_1+'_'+fuel_type_translator[filter_strings[1]]+'_'+condition_2+'_USA_QBTU.A'

    

In [ ]:
eia_base_str

In [ ]:
## Create second condition statement for cooking end use
if filter_strings[2] == 'cooking' and filter_strings[0] == 'residential':
    condition_1 = end_use_translator[filter_strings[2]]
    condition_2 = 'NA'
    
    ## 
    eia_base_str = 'AEO.2020.REF2020.CNSM_NA_'+bldg_class_translator[filter_strings[0]]+'_'+condition_1+'_'+fuel_type_translator[filter_strings[1]]+'_'+condition_2+'_USA_QBTU.A'



In [ ]:
## Create EIA series ID condition statement for other residential cases
## Same conditions as cooking, do all residential have same arrangement? do all commercial have same arrangement?
if filter_strings[0] == 'residential':
    condiiton_1 = end_use_translator[filter_strings[2]]
    condition_2 = 'NA'

In [ ]:
## Create EIA series ID condition statement for other commercial cases
if filter_strings[0] == 'commercial':
    condiiton_1 = 'NA'
    condition_2 = end_use_translator[filter_strings[2]]

In [ ]:
## Re-create constructed EIA series ID str
for bldg in  bldg_class_translator:
    if bldg_class_translator[filter_strings[0]] and end_use_translator[filter_strings[2]]:
        constructed_eia_series_id+= 'AEO.2020.REF2020.CNSM_NA_'+bldg_class_translator[filter_strings[0]]
        
        ## De-bugging
        print(constructed_eia_series_id)

In [ ]:
## Testing-9/7/20

bldg_class_translator = {'residential': 'RESD', 'commercial': 'COMM'}
end_use_translator = {'clothes washers': 'CLW', 'clothes dryers': 'CDR', 'computers': 'CMPR', 'cooking': 'CGR', 'computing': 'OTHEQPPC' ,
        'delivered energy':'DELE', 'dishwashers': 'DSW', 'fans and pumps': 'FPR', 'freezers': 'FRZ', 'lighting': 'LGHTNG', 'office equipment': 'OTHEQPNPC',
        'other uses': 'OTHU','refrigeration': 'REFR', 'cooling': 'SPC', 'heating': 'SPH', 'TVs': 'TVR', 'ventilation': 'VNTC', 'water Heating': 'WTHT' }

fuel_type_translator = {'electricity': 'ELC', 'Purchased Electricity': 'PRC', 'natural gas': 'NG'}




In [ ]:
filter_strings = ['commercial', 'natural gas', 'cooking']

In [ ]:
## Create base string structure of EIA API series id series

condition_1 = 'blank 1'
condition_2 = 'blank 2'

eia_base_str = 'AEO.2020.REF2020.CNSM_NA_'+bldg_class_translator[filter_strings[0]]+'_'+condition_1+'_'+fuel_type_translator[filter_strings[1]]+'_'+condition_2+'_USA_QBTU.A'


In [ ]:
eia_base_str

In [ ]:
### Final observations, difference btwn EIA series ID are based on bldg class
## Conditions to construct EIA API series ID should be:
if filter_strings[0] == 'residential':
    condiiton_1 = end_use_translator[filter_strings[2]]
    condition_2 = 'NA'
    eia_base_str = 'AEO.2020.REF2020.CNSM_NA_'+bldg_class_translator[filter_strings[0]]+'_'+condition_1+'_'+fuel_type_translator[filter_strings[1]]+'_'+condition_2+'_USA_QBTU.A'
    # De-bugging
    #print(eia_base_str)

In [ ]:
eia_base_str

In [ ]:
if filter_strings[0] == 'commercial':
    condiiton_1 = 'NA'
    condition_2 = end_use_translator[filter_strings[2]]
    eia_base_str = 'AEO.2020.REF2020.CNSM_NA_'+bldg_class_translator[filter_strings[0]]+'_'+condition_1+'_'+fuel_type_translator[filter_strings[1]]+'_'+condition_2+'_USA_QBTU.A'
    # De-bugging
    eia_base_str

In [ ]:
eia_base_str

In [ ]:
filter_strings[2] == end_use_translator and filter_strings[1] == fuel_type_translator:
constructed_eia_series_id = 'AEO.2020.REF2020.CNSM_NA_'+bldg_class_translator[filter_strings[0]

In [ ]:
# Converts dict to JSON string
test_data = json.dumps(test_data_dict, indent = 2)

## Outputs test data as a dict
#print(test_data_dict,type(test_data_dict))

In [ ]:
# Converts dict to list
#new_test_data = list(test_data_dict)

In [ ]:
class useful_variables(object):
    def __init__(self):
        
        ## Matches data in json file for Scout, fix format error
        self.json_data_translator_dict = {'residential': ['single family home', 'multi family home', 'mobile homes'],
                                 'electricity': ['heating','secondary heating' ,'cooling' ,'cooking','computers',
                                                 'drying','lighting', 'refrigeration','ceiling fan','fans and pumps',
                                                 'water heating','other', 'all'],
                                 'natural gas': ['heating','secondary heating', 'cooling', 'cooking', 'drying',
                                                 'water heating','other', 'all'],
                                 'distillate': ['heating', 'secondary heating', 'water heating', 'other', 'all'],
                                 'other fuel': ['heating', 'secondary heating','water heating', 'cooking', 'drying',
                                               'other', 'all'],
                                 'commercial': ['assembly', 'education', 'food sales','food service', 'health care',
                                                'lodging','small office','large office','mercantile/serice',
                                                'warehouse', 'other'], 
                                 'electricity': ['heating', 'cooling', 'ventilation', 'water heating','lighting',
                                                'refrigeration', 'cooking','PCs', 'non-PC office equipment', 'MELs','all'],
                                 'natural gas':['heating', 'cooling', 'water heating', 'cooking', 'all'],
                                 'distillate': ['heating', 'water heating', 'all']}

        ## Dict translators for building class, end use, and fuel type found on in the EIA data
        # Dict of building class
        self.bldg_class_translator = {'residential': 'RESD', 'commercial': 'COMM'}

        #Dict of end uses
        self.end_use_translator = {'Clothes washers': 'CLW', 'Clothes dryers': 'CDR', 'Computers': 'CMPR', 'Cooking': 'CGR', 'Computing': 'OTHEQPPC' ,
        'Delivered Energy':'DELE', 'Dishwashers': 'DSW', 'Fans and Pumps': 'FPR', 'Freezers': 'FRZ', 'Lighting': 'LGHTNG', 'Office Equipment': 'OTHEQPNPC',
        'Other Uses': 'OTHU','Refrigeration': 'REFR', 'Space Cooling': 'SPC', 'Space Heating': 'SPH', 'Televisions': 'TVR', 'Ventilation': 'VNTC', 'Water Heating': 'WTHT' }

        #Dict of fuel type, purchased electricity is for commercial
        self.fuel_type_translator = {'electricity': 'ELC', 'Purchased Electricity': 'PRC', 'natural gas': 'NG'}


In [ ]:
## Useful variables used in later functions
bld_class_translator = {'residential': 'RESD', 'commercial': 'COMM'}
json_data_translator_dict = {'residential': ['single family home', 'multi family home', 'mobile homes'],
                             'electricity': ['heating','secondary heating' ,'cooling' ,'cooking','computers',
                                             'drying','lighting', 'refrigeration','ceiling fan','fans and pumps',
                                             'water heating','other', 'all'],
                             'natural gas': ['heating','secondary heating', 'cooling', 'cooking', 'drying',
                                             'water heating','other', 'all'],
                             'distillate': ['heating', 'secondary heating', 'water heating', 'other', 'all'],
                             'other fuel': ['heating', 'secondary heating','water heating', 'cooking', 'drying',
                                           'other', 'all'],
                             'commercial': ['assembly', 'education', 'food sales','food service', 'health care',
                                            'lodging','small office','large office','mercantile/serice',
                                            'warehouse', 'other'], 
                             'electricity': ['heating', 'cooling', 'ventilation', 'water heating','lighting',
                                            'refrigeration', 'cooking','PCs', 'non-PC office equipment', 'MELs','all'],
                             'natural gas':['heating', 'cooling', 'water heating', 'cooking', 'all'],
                             'distillate': ['heating', 'water heating', 'all']}

fuel_type_translator = {'electricity': 'ELC', 'purchased electricity': 'PRC', 'natural gas': 'NG'}
end_use_translator = {'clothes washers': 'CLW', 'clothes dryers': 'CDR', 'computers': 'CMPR', 'cooking': 'CGR', 'computing': 'OTHEQPPC',
                      'dishwashers': 'DSW', 'fans and pumps': 'FPR', 'freezers': 'FRZ', 'lighting': 'LGHTNG', 'office equipment': 'OTHEQPNPC',
    'other uses': 'OTHU','refrigeration': 'REFR', 'cooling': 'SPC', 'heating': 'SPH', 'TVs': 'TVR', 'ventilation': 'VNTC', 'water Heating': 'WTHT' }

#print(end_use_translator)
#type(end_use_translator)

In [ ]:
## Old function
filter_strings = ['residential','electricity', 'refrigeration']
## Recursive function to traverse the json structure and grab energy values per year as numpy array
def recursive(test_data_dict, filter_strings , energy_vals, position_list=[]):
    for key, value in test_data_dict.items():
        if isinstance(value, dict):  
            recursive(value, filter_strings , energy_vals, position_list+[key]) 
            
        else:
        ## At the endpoint/leaf node, use relevant fuel type and end use combination to filter neccessary energy values, aggregate energy value by year
            
            ## Store each year as a list
            years_as_keys_list = (position_list+[key])
            
            ## Convert list into numpy array
            years_as_keys_array = np.array(years_as_keys_list)
            
            ## Extract relevant fuel type and end use combination that matches EIA data
            #filter_strings = ['residential','electricity', 'refrigeration']
            #filter_strings = ['residential','electricity', 'heating']
            
            ## Create a dict for residential and commercial bldg types 
            ## Check format in EIA for proper syntax matching, 
            all_bldg_types ={'residential': ['single family home', 'multi family home', 'mobile homes'], 
            'commercial': ['assembly', 'education', 'food sales','food service', 'health care', 'lodging',
            'small office','large office','mercantile/serice', 'warehouse', 'other']}
            
            
            ## Isolate all heating energy values by years
            #if filter_strings[1] and filter_strings[2] and all_bldg_types[filter_strings[0]] and 'energy' in years_as_keys_list:
               # print("Yay")
        
            #if any(value in years_as_keys_list for value in all_bldg_types[filter_strings[0]]) and("energy" in years_as_keys_list):
               # print("True#3")
                
            if filter_strings[1] and filter_strings[2] and all_bldg_types[filter_strings[0]] and 'energy' in years_as_keys_list:
                ## Sort by keys in year order and grab values in the order of the years as numpy array
                energy_vals+=np.array([test_data_dict[key] for key,value in sorted(test_data_dict.items())])
                #print(type(energy_vals))

    return (energy_vals)
           

In [ ]:
## Test calling recursive function, why am I getting zeros?
filter_strings = ['residential','electricity', 'refrigeration']
#filter_strings = ["electricity", "refrigeration"]
test=recursive(test_data_dict, filter_strings, np.zeros(2))
#print((test, type(test)))



In [ ]:
## All possible data combinations for bldg class, fuel type and end use
bldg_class = ['residential', 'commercial']
fuel_type = ['electricity', 'Purchased Electricity', 'natural gas']
end_use = ['clothes washers', 'clothes dryers', 'computers', 'cooking', 'computing',
'delivered energy', 'dishwashers','fans and pumps', 'freezers', 'lighting', 'office equipment',
'other uses','refrigeration', 'cooling', 'heating', 'TVs', 'ventilation', 'water heating']

## Read in data
with open('test_json_traverse.json') as f:
    data_dict = json.load(f)
                
## Construct filter strings combination to be used to call functions
for bldg in bldg_class:
    #print(str(bldg), type(bldg))
    for fuel in fuel_type:
        #print(fuel, type(fuel))
        for use in end_use:
            #print(use)
            
            ## Concatenate filter strings combination
            filter_strings = [bldg, fuel, use]
            print(filter_strings)
            
            ## Call recursive function
            test_2 = recursive(data_dict, filter_strings, np.zeros(2))
            print(test_2)

In [ ]:
class test_data_comparsion_expect_block():
    ## create an error that triggers the expect block and 
    ## Purposely input only bldg class and end use for input into recursive function
    trigger_else_block_error = ['residential','refrigeration']

    ## Expected output of else block
    expected_str_else_block = ('list index out of range, filter_strings is wrong')

    def __init__(self):
        self.data_comparsion = baseline_calculator_code_2.data_comparsion

        ## Testing the operation of the else block
        def test_else_block_error(self):
            ## Call data comparsion function,
            ## Replace filter_strings with trigger_else_block_error
            output_array_else_block = self.data_comparsion(test_data_dict, self.trigger_else_block_error, np.zeros(2))
        print(output_array_else_block) 

In [ ]:
## Testing-De-bugging: Nested for loop for filter_strings
#bldg_class = ['residential', ['single family home', 'multi family home', 'mobile homes'], 
            #'commercial',['assembly', 'education', 'food sales','food service', 'health care', 'lodging',
            #'small office','large office','mercantile/serice', 'warehouse', 'other']]
bldg_class = ['single family home', 'multi family home', 'mobile homes', 'assembly',
                'education', 'food sales','food service', 'health care','lodging','small office',
                'large office','mercantile/serice', 'warehouse', 'other']
fuel_type = ['electricity', 'Purchased Electricity', 'natural gas']
end_use = ['clothes washers', 'clothes dryers', 'computers', 'cooking', 'computing',
           'delivered energy', 'dishwashers','fans and pumps', 'freezers', 'lighting',
           'office equipment','other uses','refrigeration','cooling','heating','TVs',
           'ventilation', 'water heating']


#print( ','.join[bldg,fuel,use], type(bldg_class))
for key,value in sorted(test_data_dict.items()):
    test_keys = ([test_data_dict[key]])
    #print(test_keys, type(test_keys))
    #for list in test_keys:
        #print(list, type(list))
        #test_5 =list
        #print(test_5)   

In [ ]:
    def __init__(self):
        ## Matches data in json file for Scout, fix format error
        self.json_data_translator_dict = {'residential': ['single family home', 'multi family home', 'mobile homes'],
                                 'electricity': ['heating','secondary heating' ,'cooling' ,'cooking','computers',
                                                 'drying','lighting', 'refrigeration','ceiling fan','fans and pumps',
                                                 'water heating','other', 'all'],
                                 'natural gas': ['heating','secondary heating', 'cooling', 'cooking', 'drying',
                                                 'water heating','other', 'all'],
                                 'distillate': ['heating', 'secondary heating', 'water heating', 'other', 'all'],
                                 'other fuel': ['heating', 'secondary heating','water heating', 'cooking', 'drying',
                                               'other', 'all'],
                                 'commercial': ['assembly', 'education', 'food sales','food service', 'health care',
                                                'lodging','small office','large office','mercantile/serice',
                                                'warehouse', 'other'], 
                                 'electricity': ['heating', 'cooling', 'ventilation', 'water heating','lighting',
                                                'refrigeration', 'cooking','PCs', 'non-PC office equipment', 'MELs','all'],
                                 'natural gas':['heating', 'cooling', 'water heating', 'cooking', 'all'],
                                 'distillate': ['heating', 'water heating', 'all']}

        ## Dict translators for building class, end use, and fuel type found on in the EIA data
        # Dict of building class
        self.bld_class_translator = {'residential': 'RESD', 'commercial': 'COMM'}

        #Dict of end uses
        self.end_use_translator = {'clothes washers': 'CLW', 'clothes dryers': 'CDR', 'computers': 'CMPR', 'cooking': 'CGR', 'computing': 'OTHEQPPC' ,
        'delivered energy':'DELE', 'dishwashers': 'DSW', 'fans and pumps': 'FPR', 'freezers': 'FRZ', 'lighting': 'LGHTNG', 'office equipment': 'OTHEQPNPC',
        'other uses': 'OTHU','refrigeration': 'REFR', 'cooling': 'SPC', 'heating': 'SPH', 'TVs': 'TVR', 'ventilation': 'VNTC', 'water Heating': 'WTHT' }

        #Dict of fuel type, electricity is residential
        # purchased electricity is for commercial
        self.fuel_type_translator = {'electricity': 'ELC', 'Purchased Electricity': 'PRC', 'natural gas': 'NG'}



In [ ]:
bldg_class_translator = {'residential': 'RESD', 'commercial': 'COMM'}

end_use_translator = {'clothes washers': 'CLW', 'clothes dryers': 'CDR', 'computers': 'CMPR', 'cooking': 'CGR', 'computing': 'OTHEQPPC' ,
        'delivered energy':'DELE', 'dishwashers': 'DSW', 'fans and pumps': 'FPR', 'freezers': 'FRZ', 'lighting': 'LGHTNG', 'office equipment': 'OTHEQPNPC',
        'other uses': 'OTHU','refrigeration': 'REFR', 'cooling': 'SPC', 'heating': 'SPH', 'TVs': 'TVR', 'ventilation': 'VNTC', 'water Heating': 'WTHT' }

fuel_type_translator = {'electricity': 'ELC', 'Purchased Electricity': 'PRC', 'natural gas': 'NG'}



In [ ]:
from unittest.mock import patch, call

In [ ]:


# Text expected to be printed to the stdout by the except block
expected_except_print = ('list index out of range' )



    @patch('builtins.print')
    def test_except_print(self, mock_print):
        mock_print = baseline_calculator_code_2a.data_comparsion(data_dict, self.filter_str_exception, np.zeros(2))

        ## De-bugging purposes
        #print(mock_print)

        ## Test that printed output from the except block matches the expected string
        ## Only catches raised error if its located inside its corresponding try block
        self.assertEqual( mock_print, self.expected_except_print)
        

In [ ]:
## Testing print function

In [ ]:
import json
import numpy as np
import baseline_calculator_code_2a
import os
import requests


In [ ]:
def data_comparsion(data_dict,filter_strings, np_array):
     def __init__(self):
        ## Matches data in json file for Scout, fix format error
        self.json_data_translator_dict = {'residential': ['single family home', 'multi family home', 'mobile homes'],
                                 'electricity': ['heating','secondary heating' ,'cooling' ,'cooking','computers',
                                                 'drying','lighting', 'refrigeration','ceiling fan','fans and pumps',
                                                 'water heating','other', 'all'],
                                 'natural gas': ['heating','secondary heating', 'cooling', 'cooking', 'drying',
                                                 'water heating','other', 'all'],
                                 'distillate': ['heating', 'secondary heating', 'water heating', 'other', 'all'],
                                 'other fuel': ['heating', 'secondary heating','water heating', 'cooking', 'drying',
                                               'other', 'all'],
                                 'commercial': ['assembly', 'education', 'food sales','food service', 'health care',
                                                'lodging','small office','large office','mercantile/serice',
                                                'warehouse', 'other'], 
                                 'electricity': ['heating', 'cooling', 'ventilation', 'water heating','lighting',
                                                'refrigeration', 'cooking','PCs', 'non-PC office equipment', 'MELs','all'],
                                 'natural gas':['heating', 'cooling', 'water heating', 'cooking', 'all'],
                                 'distillate': ['heating', 'water heating', 'all']}

        ## Dict translators for building class, end use, and fuel type found on in the EIA data
        # Dict of building class
        self.bldg_class_translator = {'residential': 'RESD', 'commercial': 'COMM'}

        #Dict of end uses
        self.end_use_translator = {'clothes washers': 'CLW', 'clothes dryers': 'CDR', 'computers': 'CMPR', 'cooking': 'CGR', 'computing': 'OTHEQPPC' ,
        'delivered energy':'DELE', 'dishwashers': 'DSW', 'fans and pumps': 'FPR', 'freezers': 'FRZ', 'lighting': 'LGHTNG', 'office equipment': 'OTHEQPNPC',
        'other uses': 'OTHU','refrigeration': 'REFR', 'cooling': 'SPC', 'heating': 'SPH', 'TVs': 'TVR', 'ventilation': 'VNTC', 'water Heating': 'WTHT' }
        
        #Dict of fuel type, electricity is residential
        # purchased electricity is for commercial
        self.fuel_type_translator = {'electricity': 'ELC', 'Purchased Electricity': 'PRC', 'natural gas': 'NG'}

        # Use json_data_translator_dict to obtain the same data contained in 'val'
        # for the other climate zones and add it to the values for climate zone 1
        # to get a single list/dict/array of values by year
        try:
            with open('supporting_data/stock_energy_tech_data/mseg_res_com_cz.json') as f:
                data_dict = json.load(f)
                # call recursive function, get energy values, change filter string to match the fiter string combo you call from recursive
                recursive(data_dict, filter_strings, np.zeros(2))
                energy_vals = recursive(test_data_dict, filter_strings, np.zeros(2))

                # use json_data_translator_dict to determine the data to call from the EIA API, this is series ID
                eia_data_str = 'AEO.2020.REF2020.CNSM_NA_'+bldg_class_translator[filter_strings[0]]+'_'+end_use_translator[filter_strings[2]]+'_'+fuel_type_translator[filter_strings[1]]+'_NA_USA_QBTU.A'

                ## call EIA API and grab EIA energy values based on years
                eia_data_array, eia_data_years = data_processor(api_query('96b976d56566570e0292b914bdef21d6', eia_data_str))
                #print(type(eia_data_array))
                ## sum EIA values, compare to json energy value
                eia_energy_sum = sum(eia_data_array)
                #print(type(eia_energy_sum))
                # Test if arrays have same shape and same element values
                #if np.array_almost_equal(eia_data_array, energy_vals):
                    #print('True')

                return eia_data_array, energy_vals
        
        ## Only runs if try block is unsuccessful
        except IndexError:
            print('filter strings list index out of range')
        except AttributeError:
            print(" 'NoneType' object has no attribute 'mock_calls' ")
        ## Using bare except to catch all errors
        except:
            print('Some other error occurred')


In [ ]:
## Read in test data
with open('test_json_traverse.json') as f:
    data_dict = json.load(f)
    
    filter_str_exception = ['residential','electricity','refrigeration']
    
    mock_print = baseline_calculator_code_2a.data_comparsion(data_dict, filter_str_exception, np.zeros(2))
    print(mock_print)

In [ ]:
### Call data_comparsion function
filter_strings = ['residential','electricity', 'heating']
#print(type(filter_strings))
#data_comparison(test_data_dict, filter_strings, (np.zeros(2), np.zeros(2)))

In [ ]:
## Construct filter strings combination to be used to call functions
for bldg in bldg_class:
    print(str(bldg), type(bldg))
    for fuel in fuel_type:
        #print(fuel, type(fuel))
        for use in end_use:
            #print(use)
            ## Concatenate filter strings combination
            filter_strings = [bldg + ',' + fuel + ',' + use]
            print(filter_strings, type(filter_strings))
            ## Call data comparsion function
            data_comparsion(test_data_dict, filter_strings, (np.zeros(2), np.zeros(2))) 

            ## at the highest level of for loop, call data comparsion ; 
            ## pass a list for specify bldg ,fuel, end use 
            ## have real data dict
            ## if filter string combo present then will call function
            ## if not then will continue through iteration until next valid combo
            ## data comparsion function should continue despite valid combo, will check to make sure


In [ ]:
### My adapted version of Chioke's EX#1
from unittest.mock import patch
import baseline_calculator_code_2 as code
import unittest



In [ ]:
class test_data_comparsion_expect_block(unittest.TestCase):
    ## create an error that triggers the expect block and 
    ## Purposely input only bldg class and end use for input into recursive function
    trigger_else_block_error = ['residential','refrigeration']

    ## Expected output of else block
    expected_list_else_block = ['list index out of range, check filter_strings']

    ## Testing the operation of the else block
    @patch('baseline_calculator_code_2.data_comparsion')
    def test_else_block_error(self, recursive_error, patch_else_block):
        patch_else_block.return_value = ['list index out of range, check filter_strings']
        #recursive_error.side_effect = IndexError()
    print(type(test_else_block_error))

        ## Call data comparsion function,
        ## Replace filter_strings with trigger_else_block_error
        #output_array_else_block = self.data_comparsion(test_data_dict, self.trigger_else_block_error, np.zeros(2))
        
        ## Compare output from data comparsion function with desired output
        #self.assertalmostEqual(output_array_else_block, expected_list_else_block)
        

        


In [ ]:
### TESTING CODE BELOW

In [ ]:
### Testing data comparsion funtion's except block when pass in wrong tuple length
## Old function:


In [ ]:
## CURRENT FUNCTION

def data_comparsion(data_dict,filter_strings, np_array):
     def __init__(self):
        ## Matches data in json file for Scout, fix format error
        self.json_data_translator_dict = {'residential': ['single family home', 'multi family home', 'mobile homes'],
                                 'electricity': ['heating','secondary heating' ,'cooling' ,'cooking','computers',
                                                 'drying','lighting', 'refrigeration','ceiling fan','fans and pumps',
                                                 'water heating','other', 'all'],
                                 'natural gas': ['heating','secondary heating', 'cooling', 'cooking', 'drying',
                                                 'water heating','other', 'all'],
                                 'distillate': ['heating', 'secondary heating', 'water heating', 'other', 'all'],
                                 'other fuel': ['heating', 'secondary heating','water heating', 'cooking', 'drying',
                                               'other', 'all'],
                                 'commercial': ['assembly', 'education', 'food sales','food service', 'health care',
                                                'lodging','small office','large office','mercantile/serice',
                                                'warehouse', 'other'], 
                                 'electricity': ['heating', 'cooling', 'ventilation', 'water heating','lighting',
                                                'refrigeration', 'cooking','PCs', 'non-PC office equipment', 'MELs','all'],
                                 'natural gas':['heating', 'cooling', 'water heating', 'cooking', 'all'],
                                 'distillate': ['heating', 'water heating', 'all']}

        ## Dict translators for building class, end use, and fuel type found on in the EIA data
        # Dict of building class
        self.bld_class_translator = {'residential': 'RESD', 'commercial': 'COMM'}

        #Dict of end uses
        self.end_use_translator = {'clothes washers': 'CLW', 'clothes dryers': 'CDR', 'computers': 'CMPR', 'cooking': 'CGR', 'computing': 'OTHEQPPC' ,
        'delivered energy':'DELE', 'dishwashers': 'DSW', 'fans and pumps': 'FPR', 'freezers': 'FRZ', 'lighting': 'LGHTNG', 'office equipment': 'OTHEQPNPC',
        'other uses': 'OTHU','refrigeration': 'REFR', 'cooling': 'SPC', 'heating': 'SPH', 'TVs': 'TVR', 'ventilation': 'VNTC', 'water Heating': 'WTHT' }
        
        #Dict of fuel type, electricity is residential
        # purchased electricity is for commercial
        self.fuel_type_translator = {'electricity': 'ELC', 'Purchased Electricity': 'PRC', 'natural gas': 'NG'}

        # Use json_data_translator_dict to obtain the same data contained in 'val'
        # for the other climate zones and add it to the values for climate zone 1
        # to get a single list/dict/array of values by year
        try:
            with open('supporting_data/stock_energy_tech_data/mseg_res_com_cz.json') as f:
                data_dict = json.load(f)
                # call recursive function, get energy values, change filter string to match the fiter string combo you call from recursive
                recursive(data_dict, filter_strings, np.zeros(2))

            # IF string combo(i.e eia_data_str and json_data_translator) is present in the test_json and EIA
            # for all combos of json_data_translator and test_data; call eia for some str combos 
            #if all(value in test_data_dict for value in json_data_translator_dict.items()) and any(value in test_data_dict for value in json_data_translator_dict[eia_data_str]): 
            #eia_energy_vals+= np.array([json_data_translator_dict[key]] for key in sorted(json_data_translator_dict.items()))

        ## Only runs if try block is unsuccessful
        ## Use expection and print error, but continue until a combo is found
        
        #except AssertionError:
            #print(['First sequence is not a list, check filter_strings'])
            #pass
        except IndexError:
            print(['list index out of range, check filter_strings'])
            
        ## If try block successful, then this runs
        else:
            # use json_data_translator_dict to determine the data to call from the EIA API, this is series ID
            eia_data_str = 'AEO.2020.REF2020.CNSM_NA_'+self.bld_class_translator[filter_strings[0]]+'_'+self.end_use_translator[filter_strings[2]]+'_'+self.fuel_type_translator[filter_strings[1]]+'_NA_USA_QBTU.A'

            ## call EIA API and grab EIA energy values based on years
            eia_data_array, eia_data_years = data_getter('96b976d56566570e0292b914bdef21d6', filter_strings, eia_data_str)

            ## sum EIA values, compare to json energy value
            eia_energy_sum = sum(eia_data_array)

            ## Compare EIA and json energy values 
            ## Test if arrays have same shape and same element values
            if np.array_almost_equal(eia_data_array, energy_vals):
            #if np.array_equal(eia_data_array, energy_vals):
                print('True')
            #if eia_energy_sum == energy_vals.any():
                #print('EIA and json data are equal')
            #if eia_energy_sum != energy_vals:
                #print('EIA and json data not equal') 

            return eia_data_array, eia_data_years

        ## Close EIA API
        finally:
            print("Closing EIA API")


In [ ]:
## TESTING: Call data comparsion function with two tuples and see if I get index error
filter_strings = ['residential','electricity', 'refrigeration']

test=data_comparsion_2(test_data_dict,filter_strings, (np.zeros(2), np.zeros(2)))
print(test)

In [ ]:
## De-bugging - issue with output format of the mock_get_patcher
## Test mocking patch for EIA API call function, outputs as a dict
mock_get_patcher = (requests.get('http://api.eia.gov/series/?api_key=96b976d56566570e0292b914bdef21d6&series_id=AEO.2020.REF2020.CNSM_HHD_RESD_SPH_ELC_NA_USA_QBTU.A').json())
#mock_get_patcher
#mock_patch_dict = patch(mock_get_patcher)

## Issue with rsplit, trying solution here; this currently grabs all year and energy values,
## Need data accessed + patched line by line
## Solution 1, format data into year, energy values as a list, then patch line by line i.e. index
mock_eia_data = (mock_get_patcher['series'][0]['data'])
#type(mock_eia_data)
type(mock_eia_data)

#mock_patch_list = patch(mock_eia_data[0])
#mock_patch_list   

In [ ]:
## Chioke advice 1
## Try mocking the function call using requests.get inside api_query
import converter as converter
@patch('converter.requests.get')
def test_api_query(self, mock_api_response):
    # Set return value from mocked function
    # Mock chained call in api_query function requests.get().json() per:
    # https://docs.python.org/3/library/unittest.mock-examples.html#mocking-chained-calls
    mock_api_response.return_value.json.return_value = {'series': [{'data': ['success']}]}

    # Call function inside which the mock is applied
    func_output = converter.api_query('96b976d56566570e0292b914bdef21d6',
                                      'AEO.2020.REF2020.CNSM_HHD_RESD_SPH_ELC_NA_USA_QBTU.A')

    # Define expected output from the function
    expected_output = ['success']

    # Compare the output from the api_query function using the
    # mocked response with the expected output from the function
    self.assertListEqual(func_output, expected_output)

In [ ]:
func_output = converter.api_query('96b976d56566570e0292b914bdef21d6', 'AEO.2020.REF2020.CNSM_HHD_RESD_SPH_ELC_NA_USA_QBTU.A')
test_api_query(mock_api_response)

In [ ]:
## Try mocking the entirety of the api_query function 
## (when called inside another function)
@patch('converter.api_query')
def test_mock_entire_func(self, mock_func_return):
    ## Set return value from mocked function; test data right?
    mock_func_return.return_value = [['2016', 0.356], ['2015', 0.352]]

    ## Specify data series names in a list; api_key, filter strings for json data, EIA API series list
    ## (input to the data_getter function)
    data_names = ['residential','electricity', 'heating'],
                 

    ## Specify expected output for the given data name
    ## (outputs from the data_getter function)
    expected_arrays = [np.array([0.352, 0.356])]
    expected_years = np.array(['2015', '2016'])

    ## Call function inside which the mock is applied
    output_dict, the_years = converter.data_getter('96b976d56566570e0292b914bdef21d6',
                                                   data_names,
                                                   ['AEO.2020.REF2020.CNSM_HHD_RESD_SPH_ELC_NA_USA_QBTU.A'])

    ## Test the year output
    np.testing.assert_array_equal(the_years, expected_years)

    ## Test the dict output, iterating over data_names (the source
    ## of the key(s) for the output dict)
    for idx, nom in enumerate(data_names):
        np.testing.assert_almost_equal(output_dict[nom], expected_arrays[idx])

In [ ]:
test_mock_entire_func(mock_func_return)

In [ ]:
## Solution 2, use key, value pairs of dict , mock_get_patcher
#for key, value in mock_get_patcher.items():
    #energy_list =[]
    #if isinstance(value,dict):
        
mock_get_patcher.items()

In [ ]:
## Attempt at fixing issue with rsplit, patch EIA API and replace with test data
def test_3_access_EIA_API(api_key, series_id):
    query_str = ('http://api.eia.gov/series/?api_key=' + api_key +
                 '&series_id=' + series_id)
    mock_get_patcher = patch(requests.get.query_str.json()

    ## Start patching 'request.get' of EIA API
    mock_get = mock_get_patcher.start()

    ## Configure mock with ok status
    mock.get.return_value.ok = True

    ## Call EIA API request to server
    eia_data_array, eia_data_years = data_processor(mock_get_patcher)
    #eia_data_array, eia_data_years = data_processor(api_query('96b976d56566570e0292b914bdef21d6', eia_data_str))

    ## Stop patching 'request.get'
    mock_get_patcher.stop()

    ## Response expected if request is successfu]
    assert_is_not_none(eia_data_array, eia_data_years)

    ## Grab data from EIA API, 
    ## might not be needed, came from api query function
    try:
        data = data['series'][0]['data']
    # If an invalid series_id is used, the 'series' key will not be present
    except KeyError:
        print('\nSeries ID not available from API: ' + series_id)
    return data


In [ ]:
## Test calling mocking function
#test_3_access_EIA_API('96b976d56566570e0292b914bdef21d6', 'AEO.2020.REF2020.CNSM_HHD_RESD_SPH_ELC_NA_USA_QBTU.A')

In [ ]:
asdf = requests.get('http://api.eia.gov/series/?api_key=96b976d56566570e0292b914bdef21d6&series_id=AEO.2020.REF2020.CNSM_HHD_RESD_SPH_ELC_NA_USA_QBTU.A')

In [ ]:
asdf.json()

In [ ]:
pwd()

In [ ]:
import inspect

In [ ]:
inspect.getfile(codex)

In [ ]:
## Mock EIA API,patch function = patcher ; find source to path, start mock, stop patch 

## Import standard + third-party libraries
from unittest.mock import patch
from nose.tools import assert_is_not_none
import baseline_calculator_code_2 as codex

In [ ]:
## Reads test data json file and stores as a dict
with open('test_json_traverse.json') as f:
  test_data_dict = json.load(f)

## Using a decorator patch to identify the source of my patch as the output data from EIA API server
#@patch('baseline_calculator_code_2.eia_data_str')
#@patch('baseline_calculator_code_2.requests.get')  # might work
@patch('codex.api_query')  # might work
def test_access_EIA_API(mock_api_response):#def test_access_EIA_API():
    mock_api_response = patch('api_query')
    # Create simulated API response in the EIA API format
    mock_api_response.return_value = [['2016', 288], ['2015', 256]]
    see_mock = codex.api_query('a', 'a')
    # Generate restructured output based on mocked API response
    #eia_data_array, eia_data_years = codex.data_processor(codex.api_query('a','a'))
    return see_mock
    # -------------------------------------------------------------
    
#     ## Set up mock patch structure as the test data but the np array from recursive function
#     ## This should replace the output data of the EIA API server
#     energy_vals = recursive(test_data_dict, filter_strings, np.zeros(2))
#     mock_patcher = patch(energy_vals)
    
#     #eia_data_str = 'AEO.2020.REF2020.CNSM_NA_'+bld_class_translator[filter_strings[0]]+'_'+end_use_translator[filter_strings[2]]+'_'+fuel_type_translator[filter_strings[1]]+'_NA_USA_QBTU.A'
#     #api_key = '96b976d56566570e0292b914bdef21d6'
#     #series_id = eia_data_str
#     #query_str = ('http://api.eia.gov/series/?api_key=' + api_key +
#                  #'&series_id=' + series_id)
#     #mock_patcher = patch(data_processor(api_query('96b976d56566570e0292b914bdef21d6', eia_data_str)))
    
#     ## Start patching EIA API structure
#     mock_start_patch = mock_patcher.start()
    
#     ## Configure mock to return response with OK status
#     mock_start_patch.return_value.ok = True
    
#     ## Configure mock to return json method of results
#     mock_start_patch.return_value.json.return_value = test_mock_data_list
    
#     ## Call EIA API service, sends request to server; should replace data with test data?
#     ## Might need to check to see if it actually is
#     eia_data_array, eia_data_years = data_processor(api_query('96b976d56566570e0292b914bdef21d6', eia_data_str))

#     ## Stop patching EIA API structure
#     mock_patcher.stop()
    
#     ## If request is sent successfully, response should be returned
#     response = assert_is_not_none(eia_data_array, eia_data_years)
#     return response
    

In [ ]:
test_access_EIA_API()

In [ ]:
import baseline_calculator_code_2 as code
code.data_processor(api_query('96b976d56566570e0292b914bdef21d6', eia_data_str))

In [ ]:
test_access_EIA_API()

In [ ]:
## Testing errors from recursive function
#filter_strings = ['residential','electricity', 'heating']

## What it was:
#filter_strings[0,1]

## Changed to:
#filter_strings[1], filter_strings[2]

In [ ]:
test_data_dict

In [ ]:
## Checking list comprehension
all_bldg_types ={'residential': ['single family home', 'multi family home', 'mobile homes'], 
'commercial': ['assembly', 'education', 'food sales','food service', 'health care', 'lodging',
'small office','large office','mercantile/serice', 'warehouse', 'other']}

for key, value in test_data_dict.items():
    print(key,value)
    if any(value in test_data_dict for value in filter_strings[1]):
        print("True#1")
    
    if any(value in test_data_dict for value in filter_strings[2]):
        print("True#2")
    
    if any(value in test_data_dict for value in all_bldg_types[filter_strings[0]]) and("energy" in test_data_dict):
        print("True#3")

In [ ]:
## Looking into issues with energy_vals obtaining energy values in json
np.array([test_data_dict[key]])

In [ ]:
[[test_data_dict[key]] for key,value in sorted(test_data_dict.items())]

In [ ]:
sorted(test_data_dict.items())

In [ ]:
eia_data_array, eia_data_years = data_processor(api_query('96b976d56566570e0292b914bdef21d6', eia_data_str))
#semi_raw_eia_api_output = api_query('96b976d56566570e0292b914bdef21d6', 'AEO.2020.REF2020.CNSM_NA_RESD_CDR_ELC_NA_USA_QBTU.A')

print(eia_data_array)

In [ ]:
## Current 
filter_strings = ['residential','electricity','refrigeration']
#
eia_data_str = 'AEO.2020.REF2020.CNSM_NA_'+bld_class_translator[filter_strings[0]]+'_'+end_use_translator[filter_strings[2]]+'_'+fuel_type_translator[filter_strings[1]]+'_NA_USA_QBTU.A'
#print(eia_data_str)

In [ ]:
def data_comparison(filter_strings, json_data_translator_dict, np_array):
    # Use json_data_translator_dict to obtain the same data contained in 'val'
    # for the other climate zones and add it to the values for climate zone 1
    # to get a single list/dict/array of values by year

    try:
        filter_strings = ['residential','electricity','refrigeration']
        # call recursive function, get energy values, change filter string to match the fiter string combo you call from recursive
        recursive(json_contents, filter_strings, np.zeros(2))

        # IF string combo(i.e eia_data_str and json_data_translator) is present in the test_json and EIA
        # for all combos of json_data_translator and test_data; call eia for some str combos 
        #if all(value in test_data_dict for value in json_data_translator_dict.items()) and any(value in test_data_dict for value in json_data_translator_dict[eia_data_str]): 
        #eia_energy_vals+= np.array([json_data_translator_dict[key]] for key in sorted(json_data_translator_dict.items()))

    
    ## Only runs if try block is successful
    ## Use expection and print error, but continue until a combo is found
    except KeyError:
        print('\nBldg class, fuel type, end use combo not present json data:' + filter_strings)
        pass
    ## If try block successful, then this runs
    else:
        # use json_data_translator_dict to determine the data to call from the EIA API, this is series ID
        eia_data_str = 'AEO.2020.REF2020.CNSM_NA_'+bld_class_translator[filter_strings[0]]+'_'+end_use_translator[filter_strings[2]]+'_'+fuel_type_translator[filter_strings[1]]+'_NA_USA_QBTU.A'

        ## call EIA API and grab EIA energy values based on years
        eia_data_array, eia_data_years = data_processor(api_query('96b976d56566570e0292b914bdef21d6', eia_data_str))

        ## sum EIA values, compare to json energy value
        eia_energy_sum = sum(eia_data_array)

        return eia_data_array, eia_data_years

        #compare EIA and json energy values
        if eia_energy_sum == energy_vals:
            print('EIA and json data are equal')
        if eia_energy_sum != energy_vals:
            print('EIA and json data not equal')
        pass
    finally:
        print("Closing EIA API")
        pass

In [ ]:
data_comparison(filter_strings, json_data_translator_dict, np_array)

In [ ]:
## sum EIA values, compare to json energy value
eia_energy_sum = sum(eia_data_array)
eia_energy_sum

In [ ]:
filter_strings = ["electricity", "refrigeration"]


In [ ]:
recursive(test_data_dict, filter_strings, np.zeros(2))

In [ ]:
# # ['residential', 'electricity', 'clothes washers']
# bld_class_translator['residential']
# fuel_type_translator['electricity']
filter_strings = ['residential', 'electricity', 'clothes washers']
#print(bld_class_translator[filter_strings[0]])
#print(fuel_type_translator[filter_strings[1]])
#print(end_use_translator[filter_strings[2]])

In [ ]:
print(json_data_translator_list)

In [ ]:
print(json_data_list)

In [ ]:
json_data_list = [(k,v) for k,v in test_data_dict.items()]
json_data_translator_list = [(k,v) for k,v in json_data_translator_dict.items()]
#print (json_data_translator_list)

fuel_type_list = [(k,v) for k,v in fuel_type.items()]
print(fuel_type_list[0])

for key,value in test_data_dict.items():
    all(val in json_data_translator_list for val in )
#and any(value in test_data_dict for value in json_data_translator_dict[eia_data_str])


In [ ]:
fuel_type_translator_list = [(k,v) for k,v in fuel_type_translator.items()]
print (fuel_type_translator_list)

In [ ]:
def data_grab(data_descr_dict):
    for key,value in data_descr_dict.items():
        # if all the values in my data_list are present in my json data, then grab the energy values (i.e call the recursive function)
        if all(val in test_data_dict for val in data_descr_list[0,2]):
            #Call recursive function
            recursive()
        # also call the EIA API and grab energy values per year
        
        

#print(type(data_descr_dict)) 



In [ ]:
# example data_descr_list ['AIA_CZ1', 'residential', 'electricity' , 'clothes washers', '2015']
def data_comparison(val, data_descr_list):
    bld_class_translator = {'residential': 'RESD', 'commercial': '????'}
    end_use_translator = {'clothes washers': 'CLW', 'clothes dryers': 'CDR', ...}
    fuel_type_translator = {}
    # Use data_descr_list to obtain the same data contained in 'val'
    # for the other climate zones and add it to the values for climate zone 1
    # to get a single list/dict/array of values by year

    # use data_descr_list to determine the data to call from the EIA API
    eia_data_str = "AEO.2020.REF2020.CNSM_NA_"+bld_class_translator[]+"_"+end_use_translator[data_descr_list[2]]+"_"+fuel_type_translator[]+"_NA_USA_QBTU.A"

    # call EIA API
    stuff = converter.api_query(os.environ['EIA_API_KEY'], eia_data_str)

    # post-process 'stuff' to get values in the form you want for comparison

    # perform comparison between EIA API values and data from JSON
    # NOTE: does not need to be year-by-year, can compare entire
    # list/array/dict at once IF whatever data structures you're using
    # are ordered and the years appear in the same order
    for year in years:
        if stuff[year] != val[year]:
            print("failed", data_descr_list)  # This is probably not the best way to inform the user of what didn't match


In [ ]:
print(test_data_dict.keys)

In [ ]:
## Changes to be made to recursive function:
## (new_DONE) Rename heating_vals... want function to be generic to any case... 
## (new_DONE) Add additional arguement in the function for specific thing to extract from  years_as_keys_list
## (new_DONE) The additional agruement will be a list of other conditions to check for... i.e. cooling, heating, etc


## (old_DONE) Check placement for return, want the return to only provide the ans when the condition is met
## Make function flexible for any number of years , don't hardcore years, restructure if statement
## Inside if statement for heating, prints dict , value.. sort the dict by key(years) and 
## extract ordered set of values as a list or array and dump into np array for your values


In [ ]:
#def recursive(test_data_dict, filter_strings , energy_vals_1, energy_vals_2, all_end_uses=[],years=[],  position_list=[]):
    #for key, value in test_data_dict.items():
        if isinstance(value, dict):  # figure out how to stop just before the leaf node (or get to the leaf node and then go back one level in the dict) so that you get the dict with the years as keys
            ## Parse through JSON file 
            recursive(value,energy_vals_1, energy_vals_2, position_list+[key]) #years
            
        #else:
        ## Whatever the action is when you get to the endpoint/leaf node
            
            ## Store each year as a list
            #years_as_keys_list = (position_list+[key])
            
            ## Convert list into numpy array
            #years_as_keys_array = np.array(years_as_keys_list)
            
            ## Extract all end uses relevant un json
            #all_end_uses = years_as_keys_list[3]
            
            #filter_strings = ["heating", "electricity"]
            
            ## Isolate all heating energy values by years
            #if all(filter_strings in years_as_keys_list) and "energy" in years_as_keys_list:
                
            #if "heating" in years_as_keys_list and "energy" in years_as_keys_list: 
                ## Sort  by dict keys (i.e.  years)
                #for key, value in test_data_dict:
                    #all_years = (years+[key])
                    
                    #energy_vals[0]+= values
                    
                    ## Summed heating energy values for a given year
                    #final_energy_vals =()
                    
                    #energy_vals_1[0]+= (years,)
                    #energy_vals_1[1]+= energy_2016_vals
                    
                   # return 
                
                   # energy_2015_vals = test_data_dict.get("2015")
                   # energy_2016_vals = test_data_dict.get("2016")
                
                    ## Extract corresponding heating end_uses 
                    #heat_end_use = all_end_uses 
            
                #return print(years)  
                #return print("Summed Heating Energy Values 2015, 2016", energy_vals_1)
                #return print("End Use: Heating", heat_end_use)
            
            
            ## Isolate all refrigeration energy values for 2015 and 2016
           # if "refrigeration" in years_as_keys_list and "energy" in years_as_keys_list:
               # energy_refrig_2015_vals = test_data_dict.get("2015")
               # energy_refrig_2016_vals = test_data_dict.get("2016")
                
                ## Summed refrigeration energy values for a given year
                #energy_vals_2[0]+= energy_refrig_2015_vals
                #energy_vals_2[1]+= energy_refrig_2016_vals
                
                ## Extract corresponding refrigeration end_uses 
                #refrig_end_use = all_end_uses
                
                #return print("Summed Refrigeration Energy Values 2015, 2016", energy_vals_2)
                #return print("End Use: Refrigeration", refrig_end_use)
                
            # above line needs to be checked/modified to make sure you're
            # successfully storing the updated/modified numpy array, will add test for this
            # will add test to ensure accurate final aggreagated value
            
            return print((all_end_uses))  
                

In [ ]:
           
            ### My thoughts         
            ### EIA data can be sorted based on end_use b/c all residential , electricity, are the same
            ### End use would be i.e. heating, refrigeration, etc 
            
            
            ### Tasks
            ### Need to use the previous heating and refrigeration summing/aggregate function 
            ### To test whether or not the final value for a given year is accurate = test of aggregation
            ### Need test of modified array to ensure values are properly being added to the array
            
       
            ###Chioke's advice
            ### EIA.... structure.. value for each year... do the comparsion at the year level..
            ### energy key and dict of year values.. turn into list or numpy array of the vector of values
            ### drop the years... get values in order.. 2015, 2016, etc for each technology
            ###add in the technology lists.. after go through end uses, fuel type, for each climate zone
            ### list of 2015,2016 add up with all technology list.. should match what you would get from the EIA data
            ### Create numpy array of all energy values for all years and all technology 
            ### pre allocated numpy array of zeros, with right dimensions.. each time gets to the adding , pass the empty numpy array
            ### return the numpy array
            ### each value corresponds to the technology all years.. numpy 
        
             
            ### pre allocated np array for len 2, and then plug into if statement to parse the only 
            ### energy vals 
            
            

In [ ]:
# Calls recursive function to ouput results
recursive(test_data_dict,np.zeros(2), np.zeros(2))

In [ ]:
# Inspect the test data as a dictionary, climate zones are keys
for key in test_data_dict:
    print(key)

In [ ]:
for key in test_data_dict:
    print(key, '->', test_data_dict[key])

In [ ]:
test_data_items = test_data_dict.items()
test_data_items

In [ ]:
for item in test_data_dict.items():
    print(item)
    print(type(item))

In [ ]:
## Can use tuple unpacking to iterate through key-value pairs
for key, value in test_data_dict.items():
    print(key, '->', value)

In [ ]:
for key in test_data_dict.keys():
    print(key,'->', test_data_dict[key])

In [ ]:
# Iterate through values
for value in test_data_dict.values():
    for key in list(test_data_dict.values()):
        if key == 'heating':
            print(key)
    #print (value)

In [ ]:
key